In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=dc987ae85aa685030d856770a411a440c2320856e47fb227da32632efbaae9e0
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import IntegerType

In [ ]:
spark = SparkSession.builder.appName("MockProject").getOrCreate()

#Preparing the data

Read the csv file


In [ ]:
import pandas as pd

df_pd = pd.read_csv("/content/drive/MyDrive/Mock_project_data/train.csv")
df_pd.head()

,Id,Address,Sold Price,Summary,Type,Year built,Heating,Cooling,Parking,Lot,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,0,540 Pine Ln,3825000.0,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,...,"Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,NaN,NaN,Los Altos,94022,CA
1,1,1727 W 67th St,505000.0,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,...,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047,CA
2,2,28093 Pine Ave,140000.0,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958.0,Forced air,NaN,0 spaces,9147.0,...,NaN,49627.0,468.0,2019-08-25,180000.0,NaN,NaN,Strawberry,95375,CA
3,3,10750 Braddock Dr,1775000.0,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,1947.0,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",NaN,...,"Detached Carport, Driveway, Garage - Two Door",1775000.0,20787.0,2019-10-24,1895000.0,2016-08-30,1500000.0,Culver City,90230,CA
4,4,7415 O Donovan Rd,1175000.0,Beautiful 200 acre ranch land with several pas...,VacantLand,NaN,NaN,NaN,0 spaces,NaN,...,NaN,NaN,NaN,2019-06-07,1595000.0,2016-06-27,900000.0,Creston,93432,CA


In [ ]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47439 entries, 0 to 47438
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Id                           47439 non-null  int64  
 1   Address                      47439 non-null  object 
 2   Sold Price                   47439 non-null  float64
 3   Summary                      47085 non-null  object 
 4   Type                         47439 non-null  object 
 5   Year built                   46394 non-null  float64
 6   Heating                      40587 non-null  object 
 7   Cooling                      26745 non-null  object 
 8   Parking                      46065 non-null  object 
 9   Lot                          33258 non-null  float64
 10  Bedrooms                     44567 non-null  object 
 11  Bathrooms                    43974 non-null  float64
 12  Full bathrooms               39574 non-null  float64
 13  Total interior l

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Mock_project_data/train2.csv", sep ='\t', header=True, inferSchema=True)

In [ ]:
df.select('sold price','lot','Total interior livable area').show()

+----------+---------+---------------------------+
|sold price|      lot|Total interior livable area|
+----------+---------+---------------------------+
| 3825000.0|      1.0|                        1.0|
|  505000.0|   4047.0|                      872.0|
|  140000.0|   9147.0|                     1152.0|
| 1775000.0|     NULL|                     2612.0|
| 1175000.0|     NULL|                       NULL|
|  221000.0|   3576.0|                     1311.0|
| 1589000.0|     NULL|                       NULL|
|  480000.0|1771149.6|                     2519.0|
| 1590000.0|     NULL|                     1601.0|
| 1275000.0|  66211.2|                     2123.0|
| 1310000.0|   7570.0|                     1579.0|
|  810000.0|    653.0|                     1393.0|
| 2100000.0|   3998.0|                     3351.0|
|  565000.0|   4800.0|                     1292.0|
|  185000.0|   7662.0|                     1272.0|
| 1155500.0|     NULL|                       NULL|
| 1750000.0|     NULL|         

In [ ]:
filtered_df = df.filter(col("Total interior livable area") < 100).orderBy('Sold Price', ascending = False)
filtered_df.show()

+---+----------+----+----------+-------+-------+--------+---------+---------------------------+-------------+-----------------+---------+------------+----+---+-----+---+------------+
| Id|Sold price|type|Year built|heating|cooling|bedrooms|Bathrooms|Total interior livable area|Garage spaces|Annual tax amount|Listed on|Listed price|City|Zip|State|Age|Price per ft|
+---+----------+----+----------+-------+-------+--------+---------+---------------------------+-------------+-----------------+---------+------------+----+---+-----+---+------------+
+---+----------+----+----------+-------+-------+--------+---------+---------------------------+-------------+-----------------+---------+------------+----+---+-----+---+------------+



In [ ]:
filtered_df.count()

31

In [ ]:
house_list = ["Detached, 2 Story",
        "Detached, 4+ Story",
        "SingleFamily",
         "Detached, 3 Story",
        "4+ Story",
        "3 Level, Live/Work",
        "Townhouse",
        "3 Story, Duplex, House & Apartments, Townhouses",
        "Luxury, Townhouse",
                  "Manufactured Home",
        "Mobile Home",
        "MobileManufactured",
                      "Studio",
        "Top Floor, Studio","..."
              ]
for i in house_list:
  print(i)

Detached, 2 Story
Detached, 4+ Story
SingleFamily
Detached, 3 Story
4+ Story
3 Level, Live/Work
Townhouse
3 Story, Duplex, House & Apartments, Townhouses
Luxury, Townhouse
Manufactured Home
Mobile Home
MobileManufactured
Studio
Top Floor, Studio
...


In [ ]:

df = df.drop('Total spaces', 'Elementary School','Appliances included','Parking','Elementary School Score','Elementary School Distance','Lot','Laundry features', 'Parking features' ,'Tax assessed value', 'Full bathrooms' , 'Middle School','Middle School Score','Middle School Distance', 'High School','High School Score','High School Distance', 'Heating features','Cooling features','Flooring','Region', 'Last Sold On','Last Sold Price','Address','Summary')

central = ['force', 'radiant', 'central','infloor', 'wall', 'furnace'] #Central: Dung cho ca nha
local = ['baseboard', 'space', 'heater','wood', 'fire', 'stove','pellet'] #Dung cho mot khu vuc nhat dinh
ac = ['conditioner','ac','central'] #Central: Dung cho ca nha
local = ['fan','electric'] #Dung cho mot khu vuc nhat dinh
housing_types = {
    "Single-Family": [
        "Detached, 2 Story",
        "Detached, 4+ Story",
        "SingleFamily",
        "Top Floor, 2 Level",
        "Semi-Attached",
        "Detached, 3 Story",
        "1 Level",
        "Low-Rise (1-3), Full",
        "Ranch",
        "31",
        "4+ Story",
        "1 Level, Top Floor",
        "3 Level, Live/Work",
        "New Construction, Mid-Rise (4-8), Unit Above, 4+ Level",
        "Top Floor, 2 Level",
        "2 Story, Full",
        "Detached, Full",
        "Manufactured on Land",
        "Low-Rise (1-3)",
        "1 Level, Luxury, Low-Rise (1-3), End Unit",
        "Mid-Rise (4-8), Live/Work",
        "2 Level, Semi-Attached",
        "2 Story, Full, Attached",
        "1 Level, Unit Above, Ground Floor",
        "1 Level, Top Floor, Unit Below, Low-Rise (1-3), Full",
        "Top Floor",
    ],
    "Condominiums": [
        "Condo",
        "Mid-Rise (4-8), Unit Below, Unit Above, 3 Level",
        "1 Level, Unit Above, Ground Floor",
        "Unit Below, Low-Rise (1-3), Full, Unit Above, Attached",
        "Top Floor, Unit Below, 2 Level, Low-Rise (1-3)",
        "New Construction, Low-Rise (1-3)",
        "Residential Lot",
        "Luxury, Flat",
        "Luxury, Top Floor, Low-Rise (1-3), Detached",
        "Top Floor, Unit Below, 2 Level, Low-Rise (1-3)",
        "Top Floor, Unit Below, End Unit",
    ],
    "Townhouses": [
        "Townhouse",
        "3 Story, Duplex, House & Apartments, Townhouses",
        "Luxury, Townhouse",
        "Luxury, Townhouse, Low-Rise (1-3), Penthouse",
        "2 Story, Full, Attached",
    ],
    "Apartments": [
        "Apartment",
        "3 Story, Duplex, House & Apartments, Townhouses",
        "Hi-Rise (9+), Unit Above",
        "3 Story",
        "Hi-Rise (9+)",
        "Top Floor, Unit Below, Flat",
        "2 Houses",
        "Studio",
        "2+ RESIDENCES",
        "Mid-Rise (4-8), Live/Work",
        "OTHER",
        "1 Level, Unit Above, Ground Floor",
        "1 Level, Top Floor, Unit Below, Low-Rise (1-3), Full",
        "Duplex",
    ],
    "Mobile Homes": [
        "Manufactured Home",
        "Mobile Home",
        "MobileManufactured",
        "Manufactured on Land"
    ],
    "Tiny Homes": [
        "Studio",
        "Top Floor, Studio"
    ]
}

def is_type(heating, list_type):
  for key in list_type:
    if key in heating.lower():
      return True
  return False

def categorize_heating(heating):
  if is_type(heating,central):
    return "central"
  elif is_type(heating,local):
    return "local"
  else:
    return "other"

def categorize_cooling(cooling):
  if is_type(cooling,central):
    return "air conditioner"
  elif is_type(cooling,local):
    return "fan"
  else:
    return "other"

def categorize_house(house):
  for house_type in housing_types.keys():
    if house in housing_types[house_type]:
      return house_type

def categorize_bedroom(room):
  if (room>='0') and (room <= '9'):
    return int(room)
  else:
    return 2

def change_time(date):
  return int(date.split('-')[0])

#fill null
df = df.fillna("none", subset=["heating","cooling"])
#categorize heating
my_udf_func = udf(categorize_heating, StringType())
df = df.withColumn("heating", my_udf_func(col("heating")))
#categorize heating
my_udf_func = udf(categorize_cooling, StringType())
df = df.withColumn("cooling", my_udf_func(col("cooling")))
#drop null values
df = df.dropna(how='any')
#categorize house type
my_udf_func = udf(categorize_house, StringType())
df = df.withColumn("type", my_udf_func(col("type")))
#categorize bedrooms
my_udf_func = udf(categorize_bedroom, StringType())
df = df.withColumn("bedrooms", my_udf_func(col("bedrooms")))

#change time column
my_udf_func = udf(change_time, IntegerType())
df = df.withColumn("Listed on", my_udf_func(col("Listed on")))

#change price

df = df.withColumn("Year built", df["Year built"].cast("int"))

#add age column
df = df.withColumn("Age", 2024 -df["Year built"] )
df = df.withColumn("Price per ft", df["Sold price"]  / df["Total interior livable area"])

df = df.withColumn("Sold price", df["Sold price"].cast("double"))
df = df.withColumn("Listed price", df["Listed price"].cast("double"))

#drop outlier
df = df.filter(col("Total interior livable area") >= 500)

In [ ]:
df.show()

+---+----------+-------------+----------+-------+---------------+--------+---------+---------------------------+-------------+-----------------+---------+------------+-------------+-----+-----+---+
| Id|Sold price|         type|Year built|heating|        cooling|bedrooms|Bathrooms|Total interior livable area|Garage spaces|Annual tax amount|Listed on|Listed price|         City|  Zip|State|Age|
+---+----------+-------------+----------+-------+---------------+--------+---------+---------------------------+-------------+-----------------+---------+------------+-------------+-----+-----+---+
|  0| 3825000.0|Single-Family|      1969|central|air conditioner|       2|      0.0|                        1.0|          0.0|          12580.0|     2019|   4198000.0|    Los Altos|94022|   CA| 55|
|  1|  505000.0|Single-Family|      1926|  other|air conditioner|       3|      2.0|                      872.0|          1.0|           6253.0|     2019|    525000.0|  Los Angeles|90047|   CA| 98|
|  2|  140

In [ ]:
from pyspark.sql.functions import avg

agg_df = df.groupBy("City").agg(avg(col("Price per ft"))).alias("average price").orderBy("avg(Price per ft)", ascending = False)
agg_df.show()

+----------------+------------------+
|            City| avg(Price per ft)|
+----------------+------------------+
|          Malibu|3440.9694818526636|
|          Rialto|2224.1199530338886|
|        Atherton|1914.4173192071014|
|   Stinson Beach| 1887.055770404845|
|South Lake Tahoe| 1686.999512647772|
|   Beverly Hills| 1654.762046534476|
|   Newport Beach|1576.0183465580344|
|         Big Sur|1569.6676567868415|
|        Marshall|   1529.4823127466|
|    Pebble Beach|1465.1101552036077|
|       Palo Alto|1426.4024905982362|
|   Hermosa Beach|1338.8120801613243|
|       Los Altos|1326.2838105771698|
|      Menlo Park|  1319.91116731362|
|  Portola Valley|1245.9244436646047|
|     Carpinteria|1229.0848829615916|
|         Bolinas| 1228.108813754573|
|            Ross|1224.4744605302533|
|      Rutherford|1192.2892845970557|
|       Cupertino|1177.9926701800612|
+----------------+------------------+
only showing top 20 rows



In [ ]:
unique_values = df.select("type").distinct().collect()
print(unique_values)

[Row(type='Apartments'), Row(type='Condominiums'), Row(type='Single-Family'), Row(type='Mobile Homes'), Row(type='Townhouses'), Row(type=None)]


In [ ]:
filtered_df = df.filter(col("Listed price") < col("Sold price")).orderBy('Sold Price', ascending = False)
filtered_df.show()

+-----+----------+-------------+----------+-------+---------------+--------+---------+---------------------------+-------------+-----------------+---------+------------+-------------+-----+-----+---+------------------+
|   Id|Sold price|         type|Year built|heating|        cooling|bedrooms|Bathrooms|Total interior livable area|Garage spaces|Annual tax amount|Listed on|Listed price|         City|  Zip|State|Age|      Price per ft|
+-----+----------+-------------+----------+-------+---------------+--------+---------+---------------------------+-------------+-----------------+---------+------------+-------------+-----+-----+---+------------------+
|43398|     9.0E7|Single-Family|      1985|  other|air conditioner|       4|      6.0|                     6449.0|          1.0|           9752.0|     2013|    550000.0|       Rialto|92377|   CA| 39|13955.652039075825|
| 3674|     6.0E7|Single-Family|      2020|central|air conditioner|      10|     11.0|                    21973.0|         1

In [ ]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Sold price: double (nullable = true)
 |-- type: string (nullable = true)
 |-- Year built: integer (nullable = true)
 |-- heating: string (nullable = true)
 |-- cooling: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- Bathrooms: string (nullable = true)
 |-- Total interior livable area: string (nullable = true)
 |-- Garage spaces: double (nullable = true)
 |-- Annual tax amount: string (nullable = true)
 |-- Listed on: integer (nullable = true)
 |-- Listed price: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Age: integer (nullable = true)

